In [ ]:
# import necessary libraries
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.neural_network import MLPRegressor
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor

# read CSV file
df = pd.read_csv('data.csv')

# separate input and output columns
X = df.drop(columns=['energy_consumption', 'occupants_comfort'])
y = df[['energy_consumption', 'occupants_comfort']]

# preprocess the data
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# apply feature selection using SVM-ANOVA
svm = SVC(kernel='linear')
selector = SelectKBest(f_regression, k=5)
X_selected = selector.fit_transform(X_scaled, y)

# split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X_selected, y, test_size=0.2, random_state=42)

# define the parameter grids for grid search
param_grid_ann = {
    'hidden_layer_sizes': [(10,), (20,), (30,), (40,)],
    'learning_rate_init': [0.001, 0.01, 0.1, 1]
}

param_grid_svm = {
    'C': [0.1, 1, 10, 100],
    'gamma': [0.01, 0.1, 1]
}

param_grid_dt = {
    'max_depth': [5, 10, 15, 20],
    'min_samples_split': [2, 5, 10]
}

# create the models
ann = MLPRegressor(random_state=42)
svm = SVR()
dt = DecisionTreeRegressor(random_state=42)

# create the grid search objects
gs_ann = GridSearchCV(ann, param_grid_ann, cv=5, n_jobs=-1)
gs_svm = GridSearchCV(svm, param_grid_svm, cv=5, n_jobs=-1)
gs_dt = GridSearchCV(dt, param_grid_dt, cv=5, n_jobs=-1)

# fit the models to the training data
gs_ann.fit(X_train, y_train)
gs_svm.fit(X_train, y_train)
gs_dt.fit(X_train, y_train)

#make predictions on the test data
y_pred_ann = gs_ann.predict(X_test)
y_pred_svm = gs_svm.predict(X_test)
y_pred_dt = gs_dt.predict(X_test)

#calculate the evaluation metrics for each model
r2_ann = r2_score(y_test, y_pred_ann)
rmse_ann = np.sqrt(mean_squared_error(y_test, y_pred_ann))
mse_ann = mean_squared_error(y_test, y_pred_ann)
mae_ann = mean_absolute_error(y_test, y_pred_ann)

r2_svm = r2_score(y_test, y_pred_svm)
rmse_svm = np.sqrt(mean_squared_error(y_test, y_pred_svm))
mse_svm = mean_squared_error(y_test, y_pred_svm)
mae_svm = mean_absolute_error(y_test, y_pred_svm)

r2_dt = r2_score(y_test, y_pred_dt)
rmse_dt = np.sqrt(mean_squared_error(y_test, y_pred_dt))
mse_dt = mean_squared_error(y_test, y_pred_dt)
mae_dt = mean_absolute_error(y_test, y_pred_dt)

#print the evaluation metrics for each model
print(f'ANN: R2 = {r2_ann:.3f}, RMSE = {rmse_ann:.3f}, MSE = {mse_ann:.3f}, MAE = {mae_ann:.3f}')
print(f'SVM: R2 = {r2_svm:.3f}, RMSE = {rmse_svm:.3f}, MSE = {mse_svm:.3f}, MAE = {mae_svm:.3f}')
print(f'DT: R2 = {r2_dt:.3f}, RMSE = {rmse_dt:.3f}, MSE = {mse_dt:.3f}, MAE = {mae_dt:.3f}')

#choose the model with the best performance
if r2_ann > r2_svm and r2_ann > r2_dt:
print('ANN is the best model')
elif r2_svm > r2_ann and r2_svm > r2_dt:
print('SVM is the best model')
else:
print('DT is the best model')


import inspyred

# define the objective functions
def energy_consumption(candidate):
    return candidate[0]

def occupants_comfort(candidate):
    return candidate[1]

# define the decision variables
def generator(random, args):
    external_wall_u_value = random.uniform(0, 10)
    supply_air_temp = random.uniform(0, 30)
    return_air_temp = random.uniform(0, 30)
    supply_water_temp = random.uniform(0, 30)
    return_air_temp_chiller_heater = random.uniform(0, 30)
    air_flow = random.uniform(0, 10)
    fan_power = random.uniform(0, 10)
    chiller_power = random.uniform(0, 10)
    heater_power = random.uniform(0, 10)
    humidity = random.uniform(0, 100)
    pressure = random.uniform(0, 10)
    return [external_wall_u_value, supply_air_temp, return_air_temp, supply_water_temp,
            return_air_temp_chiller_heater, air_flow, fan_power, chiller_power, heater_power,
            humidity, pressure]

# define the constraints
def evaluator(candidates, args):
    fitness = []
    for c in candidates:
        # use the best machine learning model to predict energy consumption and occupants comfort
        energy_pred, comfort_pred = model.predict([c])
        fitness.append((energy_pred, comfort_pred))
    return fitness

# run the optimization algorithm
prng = random.Random()
prng.seed(42)

#run the optimization algorithm
ea = inspyred.ec.emo.NSGA2(prng)
ea.variator = [inspyred.ec.variators.n_point_crossover,
inspyred.ec.variators.gaussian_mutation]
ea.terminator = inspyred.ec.terminators.generation_termination
final_pop = ea.evolve(generator=generator,
evaluator=evaluator,
pop_size=100,
maximize=False,
bounder=inspyred.ec.Bounder(0, 10),
max_generations=100)

sort and print the final population
final_pop.sort(key=inspyred.ec.emo.ParetoFront.rank)
for f in final_pop:
print(f'{f.fitness[0]:.3f} {f.fitness[1]:.3f} {f.candidate}')


#Print the Pareto front of the energy consumption and occupants comfort with the ideal point
import matplotlib.pyplot as plt

# extract the fitness values from the final population
energy = [ind.fitness[0] for ind in final_pop]
comfort = [ind.fitness[1] for ind in final_pop]

# calculate the ideal point (minima of the objective functions)
ideal_energy = min(energy)
ideal_comfort = min(comfort)

# plot the Pareto front
plt.scatter(energy, comfort)
plt.plot([ideal_energy, ideal_energy], [min(comfort), max(comfort)], 'r--')
plt.plot([min(energy), max(energy)], [ideal_comfort, ideal_comfort], 'r--')
plt.xlabel('Energy consumption')
plt.ylabel('Occupants comfort')
plt.show()
